In [3]:
import os
import pandas as pd
import numpy as np
from scipy.signal import welch

def compute_ln_lf_hf(rri_series: pd.Series, fs: float = 2.0):
    """Welch PSD로 HRV 계산 후 ln(LF), ln(HF) 리턴"""
    if rri_series.isnull().any():
        rri_series = rri_series.dropna()

    if len(rri_series) < fs * 30:  # 최소 30초 이상 필요
        return np.nan, np.nan

    freqs, psd = welch(rri_series.values, fs=fs, nperseg=min(256, len(rri_series)))

    lf_band = (freqs >= 0.0033) & (freqs < 0.04)
    hf_band = (freqs >= 0.15) & (freqs <= 0.4)

    lf_power = np.trapz(psd[lf_band], freqs[lf_band])
    hf_power = np.trapz(psd[hf_band], freqs[hf_band])

    ln_lf = np.log(lf_power) if lf_power > 0 else np.nan
    ln_hf = np.log(hf_power) if hf_power > 0 else np.nan
    return ln_lf, ln_hf

def process_participant_rri_to_hrv(sensor_base_path: str, pid: str):
    """RRI → HRV 변환하여 HRV.csv 저장"""
    rri_path = os.path.join(sensor_base_path, pid, 'RRI.csv')
    hrv_path = os.path.join(sensor_base_path, pid, 'HRV.csv')

    if not os.path.exists(rri_path):
        print(f"[!] {pid} RRI.csv not found.")
        return

    try:
        # 1. RRI 데이터 불러오기
        rri_df = pd.read_csv(rri_path)
        rri_df['timestamp'] = pd.to_datetime(rri_df['timestamp'], unit='ms', utc=True)
        rri_df.set_index('timestamp', inplace=True)
        rri_df = rri_df.sort_index()

        # 2. 500ms 간격 리샘플링 + 선형 보간 (양방향)
        resampled = (
            rri_df['interval']
            .resample('500ms')
            .mean()
            .interpolate(method='linear', limit_direction='both')
        )

        # 3. 5분 단위 슬라이딩 윈도우로 HRV 계산
        features = []
        for start in pd.date_range(resampled.index[0], resampled.index[-1], freq='5min'):
            window = resampled[start : start + pd.Timedelta('5min')]
            ln_lf, ln_hf = compute_ln_lf_hf(window)
            features.append({
                'timestamp': start,
                'ln_lf': ln_lf,
                'ln_hf': ln_hf
            })

        # 4. CSV 저장
        hrv_df = pd.DataFrame(features).dropna()
        hrv_df.to_csv(hrv_path, index=False)
        print(f"[✓] {pid}: HRV.csv saved. ({len(hrv_df)} rows)")

    except Exception as e:
        print(f"[!] Error in {pid}: {e}")

def process_all_participants(sensor_base_path: str, pids=range(1, 81)):
    """P01~P80 전체 참가자 처리"""
    for i in pids:
        pid = f'P{i:02d}'
        process_participant_rri_to_hrv(sensor_base_path, pid)

# ✅ 실행 예시
sensor_path = './data/Sensor'  # 실제 경로에 맞게 조정
process_all_participants(sensor_path)


[✓] P01: HRV.csv saved. (1869 rows)
[✓] P02: HRV.csv saved. (1867 rows)
[✓] P03: HRV.csv saved. (1841 rows)
[✓] P04: HRV.csv saved. (1811 rows)
[✓] P05: HRV.csv saved. (1901 rows)
[✓] P06: HRV.csv saved. (1856 rows)
[✓] P07: HRV.csv saved. (1860 rows)
[✓] P08: HRV.csv saved. (1787 rows)
[✓] P09: HRV.csv saved. (1886 rows)
[✓] P10: HRV.csv saved. (1864 rows)
[✓] P11: HRV.csv saved. (1769 rows)
[✓] P12: HRV.csv saved. (1856 rows)
[✓] P13: HRV.csv saved. (1874 rows)
[✓] P14: HRV.csv saved. (1895 rows)
[✓] P15: HRV.csv saved. (1872 rows)
[✓] P16: HRV.csv saved. (1861 rows)
[✓] P17: HRV.csv saved. (1717 rows)
[✓] P18: HRV.csv saved. (1780 rows)
[✓] P19: HRV.csv saved. (1851 rows)
[✓] P20: HRV.csv saved. (1873 rows)
[✓] P21: HRV.csv saved. (1862 rows)
[✓] P22: HRV.csv saved. (473 rows)
[✓] P23: HRV.csv saved. (1863 rows)
[✓] P24: HRV.csv saved. (1864 rows)
[✓] P25: HRV.csv saved. (1141 rows)
[✓] P26: HRV.csv saved. (1874 rows)
[!] P27 RRI.csv not found.
[✓] P28: HRV.csv saved. (1881 rows)
[✓

In [10]:
import os

def delete_hrv_files(sensor_base_path: str, pids=range(1, 81)):
    """P01~P80 폴더 내 HRV.csv 파일 삭제"""
    for i in pids:
        pid = f'P{i:02d}'
        hrv_path = os.path.join(sensor_base_path, pid, 'HRV.csv')
        
        if os.path.exists(hrv_path):
            try:
                os.remove(hrv_path)
                print(f"[🗑] {pid}: HRV.csv deleted.")
            except Exception as e:
                print(f"[!] Failed to delete {pid}: {e}")
        else:
            print(f"[ ] {pid}: HRV.csv not found. Skipped.")

# ✅ 실행
sensor_path = './data/Sensor'  # 경로를 실제 사용 중인 위치로 수정 가능
delete_hrv_files(sensor_path)

[🗑] P01: HRV.csv deleted.
[ ] P02: HRV.csv not found. Skipped.
[ ] P03: HRV.csv not found. Skipped.
[ ] P04: HRV.csv not found. Skipped.
[ ] P05: HRV.csv not found. Skipped.
[ ] P06: HRV.csv not found. Skipped.
[ ] P07: HRV.csv not found. Skipped.
[ ] P08: HRV.csv not found. Skipped.
[ ] P09: HRV.csv not found. Skipped.
[ ] P10: HRV.csv not found. Skipped.
[ ] P11: HRV.csv not found. Skipped.
[ ] P12: HRV.csv not found. Skipped.
[ ] P13: HRV.csv not found. Skipped.
[ ] P14: HRV.csv not found. Skipped.
[ ] P15: HRV.csv not found. Skipped.
[ ] P16: HRV.csv not found. Skipped.
[ ] P17: HRV.csv not found. Skipped.
[ ] P18: HRV.csv not found. Skipped.
[ ] P19: HRV.csv not found. Skipped.
[ ] P20: HRV.csv not found. Skipped.
[ ] P21: HRV.csv not found. Skipped.
[ ] P22: HRV.csv not found. Skipped.
[ ] P23: HRV.csv not found. Skipped.
[ ] P24: HRV.csv not found. Skipped.
[ ] P25: HRV.csv not found. Skipped.
[ ] P26: HRV.csv not found. Skipped.
[ ] P27: HRV.csv not found. Skipped.
[ ] P28: HRV

In [11]:
import os
import pandas as pd
import numpy as np
from scipy.signal import welch

def compute_ln_lf_hf(rri_series: pd.Series, fs: float = 2.0):
    """Welch PSD로 HRV 계산 후 ln(LF), ln(HF) 리턴 (신뢰성 검사 포함)"""
    rri_series = rri_series.dropna()

    if len(rri_series) < fs * 30:
        return np.nan, np.nan
    if rri_series.std() < 1.0:
        return np.nan, np.nan

    freqs, psd = welch(rri_series.values, fs=fs, nperseg=min(256, len(rri_series)))

    lf_band = (freqs >= 0.0033) & (freqs < 0.04)
    hf_band = (freqs >= 0.15) & (freqs <= 0.4)

    lf_power = np.trapz(psd[lf_band], freqs[lf_band])
    hf_power = np.trapz(psd[hf_band], freqs[hf_band])

    if lf_power < 1e-6 or hf_power < 1e-6:
        return np.nan, np.nan

    ln_lf = np.log(lf_power)
    ln_hf = np.log(hf_power)
    return ln_lf, ln_hf


def process_participant_rri_to_hrv(sensor_base_path: str, pid: str):
    rri_path = os.path.join(sensor_base_path, pid, 'RRI.csv')
    hrv_path = os.path.join(sensor_base_path, pid, 'HRV.csv')

    if not os.path.exists(rri_path):
        print(f"[!] {pid} RRI.csv not found.")
        return

    try:
        rri_df = pd.read_csv(rri_path)
        rri_df['timestamp'] = pd.to_datetime(rri_df['timestamp'], unit='ms', utc=True)
        rri_df.set_index('timestamp', inplace=True)
        rri_df = rri_df.sort_index()

        resampled = (
            rri_df['interval']
            .resample('500ms')
            .mean()
            .interpolate(method='linear', limit_direction='both')
        )

        features = []
        for start in pd.date_range(resampled.index[0], resampled.index[-1], freq='5min'):
            window = resampled[start : start + pd.Timedelta('5min')]
            ln_lf, ln_hf = compute_ln_lf_hf(window)
            if not np.isnan(ln_lf) and not np.isnan(ln_hf):
                features.append({
                    'timestamp': start,
                    'ln_lf': ln_lf,
                    'ln_hf': ln_hf
                })

        hrv_df = pd.DataFrame(features)

        # ✅ timestamp를 epoch milliseconds로 변환
        hrv_df['timestamp'] = hrv_df['timestamp'].astype(np.int64) // 10**6

        hrv_df.to_csv(hrv_path, index=False)
        print(f"[✓] {pid}: HRV.csv saved. ({len(hrv_df)} valid rows)")

    except Exception as e:
        print(f"[!] Error in {pid}: {e}")


def process_all_participants(sensor_base_path: str, pids=range(1, 81)):
    for i in pids:
        pid = f'P{i:02d}'
        process_participant_rri_to_hrv(sensor_base_path, pid)


# ✅ 실행
sensor_path = './data/Sensor'
process_all_participants(sensor_path)


[✓] P01: HRV.csv saved. (717 valid rows)
[✓] P02: HRV.csv saved. (786 valid rows)
[✓] P03: HRV.csv saved. (385 valid rows)
[✓] P04: HRV.csv saved. (800 valid rows)
[✓] P05: HRV.csv saved. (716 valid rows)
[✓] P06: HRV.csv saved. (651 valid rows)
[✓] P07: HRV.csv saved. (721 valid rows)
[✓] P08: HRV.csv saved. (441 valid rows)
[✓] P09: HRV.csv saved. (807 valid rows)
[✓] P10: HRV.csv saved. (1048 valid rows)
[✓] P11: HRV.csv saved. (627 valid rows)
[✓] P12: HRV.csv saved. (827 valid rows)
[✓] P13: HRV.csv saved. (730 valid rows)
[✓] P14: HRV.csv saved. (745 valid rows)
[✓] P15: HRV.csv saved. (983 valid rows)
[✓] P16: HRV.csv saved. (1103 valid rows)
[✓] P17: HRV.csv saved. (511 valid rows)
[✓] P18: HRV.csv saved. (629 valid rows)
[✓] P19: HRV.csv saved. (456 valid rows)
[✓] P20: HRV.csv saved. (954 valid rows)
[✓] P21: HRV.csv saved. (973 valid rows)
[✓] P22: HRV.csv saved. (216 valid rows)
[✓] P23: HRV.csv saved. (891 valid rows)
[✓] P24: HRV.csv saved. (727 valid rows)
[✓] P25: HRV.c